In [1]:
def sir_11(X, y, num_slices, K):
    X = X - np.mean(X, axis = 0)
    n_samples, n_features = X.shape
    V_hat = np.zeros([X.shape[1], X.shape[1]])
    # Step 1: Sort the data by the response variable
    sorted_indices = np.argsort(y)
    X_sorted = X[sorted_indices]
    y_sorted = y[sorted_indices]
    # Step 2: Divide the data into slices
    slice_size = n_samples // num_slices
    ph_hat = slice_size/n_samples
    slices = []
    for i in range(num_slices):
        start_idx = i * slice_size
        if i < num_slices - 1:
            end_idx = (i + 1) * slice_size
        else:  # Last slice includes any remaining samples
            end_idx = n_samples
        slices.append((X_sorted[start_idx:end_idx], y_sorted[start_idx:end_idx]))
    
    # Step 3: Compute the means of the predictors within each slice
    X_means = np.array([np.mean(slice_X, axis=0) for slice_X, _ in slices])
    
    # Step 4: Center the predictor means
    # X_centered = (X_means - np.mean(X, axis=0))
    X_centered = X_means - np.mean(X, axis=0)
    
    V_hat = np.add(V_hat,ph_hat * np.matmul(X_centered.T, X_centered))
    min_val = np.min(V_hat)
    max_val = np.max(V_hat)
    V_hat = V_hat / max_val
    
    eigenvalues, eigenvectors = np.linalg.eig(V_hat)
    K_index = np.argpartition(np.abs(eigenvalues), X.shape[1]-K) >= X.shape[1]-K
    K_largest_eigenvectors = eigenvectors[:, K_index]
    edr_est =  K_largest_eigenvectors
    return edr_est, V_hat

In [2]:
#add risk to be averaged over replicas, add prediction error for obj2 for averaged direction
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
def ave(arr, N):
    for i in range(len(arr)):
        arr[i] = arr[i]/N 
    return arr 
def compute_eigen(Q4, P, K):
    eigenvalues2, eigenvectors2 = np.linalg.eig(Q4)
    K_index = np.argpartition(np.abs(eigenvalues2), P - K) >= P - K
    K_largest_eigenvectors = eigenvectors2[:, K_index]
    edr_est = K_largest_eigenvectors  
    if edr_est[0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    return edr_est
    
def proj(edr_est): 
    E = edr_est @ np.linalg.inv(edr_est.T @ edr_est) @ edr_est.T
    return E

def exhi(obj1):        
    array1 = np.vectorize(lambda x: f"{x:.6f}")(obj1)
    table = tabulate(array1, tablefmt='latex_raw')
    # Split the table into lines
    lines = table.split('\n')
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    print(latex_table)
    
def MSE(X, y):
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate errors
    mse = mean_squared_error(y_test, y_pred)
    return mse

def TestAR2(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep):
    ARnum = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; p = 2; P1 = 2*P; AR = initial_value
    noise1 = np.zeros((num_N, n_gene))
    noise2 = np.zeros((num_N, n_obs+S+3))
    y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
    X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
    obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
    
    prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]
    
    prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
    M = [np.zeros((P, 1)) for _ in range(S)] 
    proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
    proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]
    
    proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
    # error_1_1 is for dim 2p, error_1_2 is for dim p
    error_1_1 = [np.zeros((1, )) for _ in range(S)]
    error_1_2 = [np.zeros((1, )) for _ in range(S)]
    
    error_2 = [np.zeros((1, )) for _ in range(S)]
    g = np.zeros((S, 1))
    g1 = np.zeros((S, 1))
    g2 = np.zeros((S, 1))
    projection_1_norm = np.zeros((S, 1))
    projection_2_norm = np.zeros((S, 1))
    # True_projection for 2p and p
    True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
    True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))
    
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    obj2_ave = np.zeros((P1, 1))

    error_1_3 = 0
    prediction_mse_1_3 = 0
    proj_error_1_3 = 0
    
    exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    while n1 < n_rep:  
        obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
        obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
        # Data Generation
        for h in range(num_N):
            noise1[h] = np.random.normal(0, 1, size=(n_gene)) 
            noise2[h] = np.random.normal(0, 1, size=(n_obs+S+3))
            # noise[h] = np.random.normal(0, 0.5, size=(n_obs+S))
        ar_series = np.zeros((num_N, n_obs+S+3))
        # change initial value and burn-off?
        ############################################################### 
        for t in range(1, n_gene + 1):
            for i in range(4):
                AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
                AR[i][1] = AR[i][0]
                AR[i][0] = AR_temp
        for i in range(4):
            for k in range(p):
                ar_series[i][k] = AR[i][k]
        for t in range(2, n_obs+S+3):
            for i in range(4):
                ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]
        
        for a in range(2, S+2):
            y[a-2] = ar_series[4][a:n_obs+a]
            X1[a-2] = np.concatenate((np.array([ar_series[i][a:n_obs+a] for i in range(P)]), np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)])), axis = 0).T #.reshape(-1,1)
        # X = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.array([ar_series[i][1:n_obs+1] for i in range(P)])), axis = 0).T
        ################################################################
        # original data generation
        # for t in range(0, n_obs+S):
        #     ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
        #     ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
        #     ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
        #     ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
        #     ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        # for a in range(0, S):
        #     y[a] = ar_series[4][a:n_obs+a]
        #     X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1, 1) for i in range(P)], axis = 1)
        X = X1[0]
        V1 = []
        for a in range(0, S):
            _, M = sir_11(X, y[a], H, K)
            V1.append(M)
    # objective 1: experiment
        X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
        for q in range(0, S):
            phi = ar_coeff[0] + ar_coeff[1]
            Q3 = np.zeros((P1, P1))
            for j in range(P1):
                for k in range(P1):
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T)) @ V1[a] @ np.linalg.inv(np.cov(X.T)))[j, k] * (phi[k] ** a) for a in range(0, q + 1))
            
            # Method 1: dimension 2p 
            edr_est = compute_eigen(Q3, P1, K)
            if edr_est[0] < 0:
                edr_est = -edr_est
            for i in range(ARnum):
                edr_est[i*P] = edr_est[i*P]/ar_coeff[i][0]
                edr_est[i*P + 1] = edr_est[i*P + 1]/ar_coeff[i][1]
            edr_est = np.real(edr_est / np.linalg.norm(edr_est))
            # error_1[q] += np.mean([abs(edr_est[0] / edr_est[1] - a1/a2), abs(edr_est[P] / edr_est[P + 1] - a1/a2)])
            prediction_mse_1_1[q] += MSE(X @ edr_est, y[0]) 
            proj_error_1_1[q] += np.linalg.norm((proj(edr_est) - proj(True_projection_2p)), 'fro')**2
            
            # Method 2: additional step, dimension p, average every p dimensional vector. 
            average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
            error_1_2[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
            prediction_mse_1_2[q] += MSE((X2 @ average_edr_est).reshape(-1,1), y[0])
            proj_error_1_2[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
        
        # Method 3: q = 0 use cross-sectional data X_t y_t, q = 0
        edr_est_3, _ = sir_11(X2, y[0], H, K)
        edr_est_3 = edr_est_3 @ np.linalg.inv(np.cov(X.T))
        if edr_est_3[0] < 0:
            edr_est_3 = -edr_est_3
        edr_est_3 = np.real(edr_est_3 / np.linalg.norm(edr_est_3))
        error_1_3 += abs(edr_est_3[0] / edr_est_3[1] - a1/a2)
        prediction_mse_1_3 += MSE(X2 @ edr_est_3, y[0])
        proj_error_1_3 += np.linalg.norm((proj(edr_est_3) - proj(True_projection_p)), 'fro')**2
                
        n1 += 1  
        
    # error_1_1 = ave(error_1_1, n_rep)    
    error_1_2 = ave(error_1_2, n_rep)
    error_1_3 = error_1_3/n_rep
    prediction_mse_1_1 = ave(prediction_mse_1_1, n_rep)
    prediction_mse_1_2 = ave(prediction_mse_1_2, n_rep)
    prediction_mse_1_3 = prediction_mse_1_3/n_rep
    proj_error_1_1 = ave(proj_error_1_1, n_rep)
    proj_error_1_2 = ave(proj_error_1_2, n_rep)       
    proj_error_1_3 = proj_error_1_3/n_rep
    
    index_array = list(range(len(obj_1)))
    for i in range(S):
        exhi_1[i] = prediction_mse_1_1[i]
        exhi_1[i] = np.vstack((exhi_1[i], proj_error_1_1[i]))
        exhi_1[i] = np.vstack((np.array([[index_array[i]]]), exhi_1[i]))
    
    for i in range(S):
        exhi_2[i] = error_1_2[i]
        exhi_2[i] = np.vstack((exhi_2[i], proj_error_1_2[i]))
        exhi_2[i] = np.vstack((exhi_2[i], prediction_mse_1_2[i]))
        exhi_2[i] = np.vstack((np.array([[index_array[i]]]), exhi_2[i]))
    
    #     obj_1[i] = np.vstack((obj_1[i], g[i].reshape(1,-1)))
    #     obj_1[i] = np.vstack((np.array([[index_array[i]]]), obj_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), proj_error_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), mse[i]))
      
    #objective 1: exhibition
    # for i in range(S):
    #     g[i] = abs(obj_1[i][0] / obj_1[i][1] - a1/a2)
    #     obj_1[i] = np.vstack((obj_1[i], g[i].reshape(1,-1)))
    #     obj_1[i] = np.vstack((np.array([[index_array[i]]]), obj_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), proj_error_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), mse[i]))
        
    # Result1 = [g[0] - min(g), projection1_norm[0] - min(projection1_norm), mse[0] - min(mse)]
    # Result1 = [x.item() for x in Result1]
        
    #objective 2: exhibition
    
    # for i in range(S):
    #     g[i] = abs(obj2[i][0] / obj2[i][1] - a1/a2)
    #     projection2_norm[i] = np.linalg.norm(proj2[i] - proj(True_projection), 'fro')
    #     obj2[i] = np.vstack((obj_2_avevec[i], g[i].reshape(1,-1)))
    #     obj2[i] = np.vstack((np.array([[index_array[i]]]), obj_2_avevec[i]))    
    #     obj2[i] = np.vstack((np.array(obj2[i]), projection2_norm[i]))
    #     obj2[i] = np.vstack((np.array(obj2[i]), mse2[i]))
    #     obj2[i] = np.vstack((np.array(obj2[i]), abs(obj_2_avevec[i][0]/obj_2_avevec[i][1] - 2/3)))
    #     obj2[i] = np.vstack((np.array(obj2[i]), np.linalg.norm(proj(obj_2_avevec[i]) - proj(True_projection), 'fro')))
    #     obj2[i] = np.vstack((np.array(obj2[i]), Mse1(X @ obj_2_avevec[i], y[0])))
    
    # for i in range(S):
    #     g[i] = abs(obj_2_avevec[i][0] / obj_2_avevec[i][1] - a1/a2)
    #     g1[i] = proj_error_2[i]
    #     g2[i] = Mse1(X @ obj_2_avevec[i], y[0])
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g[i].reshape(1,-1)))
    #     obj_2_avevec[i] = np.vstack((np.array([[index_array[i]]]), obj_2_avevec[i]))    
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g1[i]))
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g2[i]))
    
    # Result2 = [g[0] - min(g), g1[0] - min(g1), g2[0] - min(g2)] 
    # Result2 = [x.item() for x in Result2]
    
    # print("error_1:",[x.item() for x in error_1])
    # print("error_2:",[x.item() for x in error_2])
    # print("proj_error_1:",[x.item() for x in proj_error_1]) 
    # print("proj_error_2:",[x.item() for x in proj_error_2])
    
    # print(obj_2_avevec)
    # print(obj_2)
    # print("Obj1")
    # exhi(obj1) 
    # print("Obj2")
    # exhi(obj_2_avevec)
    # print(Result1), print(Result2)
    # return obj_2_avevec
    print("Obj1 Method 1")
    exhi(exhi_1)
    print("Obj1 Method 2")
    exhi(exhi_2)
    # print("Obj1 Method 3")
    return error_1_3, proj_error_1_3, prediction_mse_1_3

# $X_{t}, X_{t-1}, \text{Objective 1}, AR(p)$

## True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])

In [38]:
ar_coeff = [[0.3, 0.5, 0.5, 0.6], [0.4, 0.45, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 1;
TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.593239 & 0.488733 & 0.053653 \\ \hline
 1 & 0.593604 & 0.489316 & 0.053711 \\ \hline
 2 & 0.59371  & 0.489493 & 0.053726 \\ \hline
 3 & 0.593743 & 0.489556 & 0.053732 \\ \hline
 4 & 0.593757 & 0.489577 & 0.053733 \\ \hline
 5 & 0.593758 & 0.48958  & 0.053734 \\ \hline
 6 & 0.593759 & 0.48958  & 0.053734 \\ \hline
 7 & 0.593759 & 0.48958  & 0.053734 \\ \hline
 8 & 0.593759 & 0.48958  & 0.053734 \\ \hline
 9 & 0.593759 & 0.48958  & 0.053734 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.04011476]
0.0018776456318951279
0.00980676513768034


In [31]:
ar_coeff = [[0.3, 0.2, 0.5, 0.6], [0.5, 0.45, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.356871 & 0.168917 & 0.034961 \\ \hline
 1 & 0.35493  & 0.168519 & 0.034975 \\ \hline
 2 & 0.356773 & 0.169868 & 0.035165 \\ \hline
 3 & 0.357277 & 0.170237 & 0.035219 \\ \hline
 4 & 0.357436 & 0.170373 & 0.035238 \\ \hline
 5 & 0.357486 & 0.170419 & 0.035245 \\ \hline
 6 & 0.357498 & 0.170434 & 0.035248 \\ \hline
 7 & 0.357502 & 0.17044  & 0.035249 \\ \hline
 8 & 0.357503 & 0.170441 & 0.035249 \\ \hline
 9 & 0.357503 & 0.170442 & 0.035249 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.31420367]
0.13142531264317867
0.02981173229947657


In [32]:
ar_coeff = [[0.2, 0.3, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.116248 & 0.079598 & 0.024136 \\ \hline
 1 & 0.119268 & 0.08674  & 0.025362 \\ \hline
 2 & 0.119752 & 0.088505 & 0.025676 \\ \hline
 3 & 0.119784 & 0.088992 & 0.025765 \\ \hline
 4 & 0.119786 & 0.089127 & 0.025791 \\ \hline
 5 & 0.119786 & 0.089169 & 0.025798 \\ \hline
 6 & 0.119786 & 0.089182 & 0.025801 \\ \hline
 7 & 0.119786 & 0.089187 & 0.025802 \\ \hline
 8 & 0.119786 & 0.089189 & 0.025802 \\ \hline
 9 & 0.119786 & 0.089189 & 0.025802 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.0778975]
0.05041126751092993
0.020169101044991568


In [33]:
ar_coeff = [[0.01, 0.01, 0.01, 0.01], [0.01, 0.01, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 4.05714 & 1.44157 & 0.104601 \\ \hline
 1 & 4.06384 & 1.44158 & 0.104602 \\ \hline
 2 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 3 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 4 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 5 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 6 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 7 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 8 & 4.06385 & 1.44158 & 0.104602 \\ \hline
 9 & 4.06385 & 1.44158 & 0.104602 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.04055842]
0.006483091799923092
0.010465864952303672


In [145]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.043348 & 0.011777 & 0.012876 \\ \hline
 1 & 0.085194 & 0.017646 & 0.013668 \\ \hline
 2 & 0.118424 & 0.022633 & 0.014312 \\ \hline
 3 & 0.142088 & 0.027183 & 0.014953 \\ \hline
 4 & 0.157656 & 0.030523 & 0.015452 \\ \hline
 5 & 0.166475 & 0.032528 & 0.015753 \\ \hline
 6 & 0.171512 & 0.033723 & 0.015932 \\ \hline
 7 & 0.174283 & 0.034401 & 0.016035 \\ \hline
 8 & 0.175605 & 0.034726 & 0.016084 \\ \hline
 9 & 0.176282 & 0.034891 & 0.016109 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.103833]
0.056515814603976304
0.022319065813024074


In [150]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 50000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.04672  & 0.011171 & 0.01248  \\ \hline
 1 & 0.081085 & 0.01535  & 0.013263 \\ \hline
 2 & 0.112292 & 0.019938 & 0.013962 \\ \hline
 3 & 0.13579  & 0.024365 & 0.014658 \\ \hline
 4 & 0.15147  & 0.027803 & 0.015199 \\ \hline
 5 & 0.160602 & 0.029861 & 0.01552  \\ \hline
 6 & 0.165836 & 0.031094 & 0.015716 \\ \hline
 7 & 0.168681 & 0.031782 & 0.015826 \\ \hline
 8 & 0.169981 & 0.032104 & 0.015878 \\ \hline
 9 & 0.170617 & 0.032261 & 0.015903 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.10542507]
0.07241548619068974
0.025875411850980013


In [123]:
# Using the Augmented Dickey-Fuller (ADF) test
from statsmodels.tsa.stattools import adfuller
def check_stationarity(post_burn_samples):
    result = adfuller(post_burn_samples)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    return result[1] < 0.05  # Stationary if p-value < 0.05
    
def TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep):
    p = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; n_burn_round = 2; P1 = p*P; AR = initial_value 
    noise1 = np.zeros((num_N, n_burn))
    noise2 = np.zeros((num_N, n_obs+S+3))
    y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
    X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
    obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
    prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
    M = [np.zeros((P, 1)) for _ in range(S)] 
    proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
    proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]
    proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
    
    # error_1_1 is for dim 2P, error_1_2 is for dim P
    error_1_1 = [np.zeros((1, )) for _ in range(S)]
    error_1_2 = [np.zeros((1, )) for _ in range(S)]
    error_2 = [np.zeros((1, )) for _ in range(S)]
    g = np.zeros((S, 1))
    g1 = np.zeros((S, 1))
    g2 = np.zeros((S, 1))
    projection_1_norm = np.zeros((S, 1))
    projection_2_norm = np.zeros((S, 1))
    # True_projection for 2P and P
    True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
    # True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
    True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))
    
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    obj2_ave = np.zeros((P1, 1))
    error_1_3 = 0
    prediction_mse_1_3 = 0
    proj_error_1_3 = 0
    exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    while n1 < n_rep:
        # Data Generation
        for h in range(num_N):
            noise1[h] = np.random.normal(0, 0.1, size=(n_burn)) 
            noise2[h] = np.random.normal(0, 0.1, size=(n_obs+S+3))
            # noise[h] = np.random.normal(0, 0.5, size=(n_obs+S))
        ar_series = np.zeros((num_N, n_obs+S+3))
        # change initial value and burn-off?
        ############################################################### 
        for t in range(1, n_burn + 1):
            for i in range(4):
                AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
                AR[i][1] = AR[i][0]
                AR[i][0] = AR_temp
        for i in range(4):
            for k in range(n_burn_round):
                ar_series[i][k] = AR[i][k]
        for t in range(2, n_obs+S+3):
            for i in range(4):
                ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]
        for a in range(2, S+2):
            y[a-2] = ar_series[4][a:n_obs+a]
            X1[a-2] = np.concatenate((np.array([ar_series[i][a:n_obs+a] for i in range(P)]), np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)])), axis = 0).T 
        X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.zeros((P, n_obs))), axis = 0).T
        X3 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
        phi = ar_coeff[0] + ar_coeff[1]
        X = X1[0]
        V1 = []
        for a in range(0, S):
            if a == 0:
                _, M = sir_11(X2, y[a], H, K)
                V1.append(M)
            else:    
                _, M = sir_11(X, y[a], H, K)
                V1.append(M)
        # objective 1: experiment
        zero_1 = np.zeros((P, (p-1)*P))  # Shape (P, (p-1)*P)
        zero_2 = np.zeros(((p-1)*P, P))  # Shape ((p-1)*P, P)
        zero_3 = np.zeros(((p-1)*P, (p-1)*P))  # Shape ((p-1)*P, (p-1)*P)
        
        # Assemble the block matrix
        top_block = np.hstack([np.linalg.inv(np.cov(X3.T)), zero_1])  # Top block
        bottom_block = np.hstack([zero_2, zero_3])  # Bottom block
        
        Mhat =  np.vstack([top_block, bottom_block]) 
        
        for q in range(0, S):
            Q3 = np.zeros((P1, P1))
            for j in range(P1):
                for k in range(P1):
                    # np.linalg.inv(np.cov(X2.T)) is invalid, have to use np.linalg.pinv(np.cov(X2.T))??
                    Q3[j, k] = (Mhat @ V1[0] @ Mhat)[j, k] + sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T)) @ V1[a] @ np.linalg.inv(np.cov(X.T)))[j, k] * (phi[k] ** a) for a in range(1, q + 1))
            # Method 1: dimension 2p 
            edr_est = compute_eigen(Q3, P1, K)
            # if edr_est[0] < 0:
            #     edr_est[0:P] = -edr_est[0:P]
            # if edr_est[P] < 0:
            #     edr_est[P:] = -edr_est[P:]
            # X_t, X_{t-1}, X_{t-1} should divide ar coeff
            for i in range(1, p):
                for j in range(P):
                    edr_est[i*P + j] = edr_est[i*P + j]/ar_coeff[i-1][j]
            edr_est = np.real(edr_est / np.linalg.norm(edr_est))
            error_1_1[q] += abs(edr_est[0] / edr_est[1] - a1/a2)
            prediction_mse_1_1[q] += MSE(X @ edr_est, y[0]) 
            proj_error_1_1[q] += np.linalg.norm((proj(edr_est) - proj(True_projection_2p)), 'fro')**2
            
            # Method 2: additional step, dimension P, average every P dimensional vector. 
            average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
            error_1_2[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
            prediction_mse_1_2[q] += MSE((X3 @ average_edr_est).reshape(-1,1), y[0])
            proj_error_1_2[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
            
        # Method 3: q = 0 use cross-sectional data X_t y_t, q = 0
        edr_est_3, M1 = sir_11(X3, y[0], H, K)
        # edr_est_3 = np.linalg.inv(np.cov(X3.T)) @ edr_est_3 
        edr_est_3 = compute_eigen(np.linalg.inv(np.cov(X3.T)) @ M1 @ np.linalg.inv(np.cov(X3.T)), P, K)
        if edr_est_3[0] < 0:
            edr_est_3 = -edr_est_3
        edr_est_3 = np.real(edr_est_3 / np.linalg.norm(edr_est_3))
        error_1_3 += abs(edr_est_3[0] / edr_est_3[1] - a1/a2)
        prediction_mse_1_3 += MSE(X3 @ edr_est_3, y[0])
        proj_error_1_3 += np.linalg.norm((proj(edr_est_3) - proj(True_projection_p)), 'fro')**2
        
        n1 += 1
    index_array = list(range(len(obj_1)))
    
    error_1_1 = ave(error_1_1, n_rep)    
    error_1_2 = ave(error_1_2, n_rep)
    error_1_3 = error_1_3/n_rep
    prediction_mse_1_1 = ave(prediction_mse_1_1, n_rep)
    prediction_mse_1_2 = ave(prediction_mse_1_2, n_rep)
    prediction_mse_1_3 = prediction_mse_1_3/n_rep
    proj_error_1_1 = ave(proj_error_1_1, n_rep)
    proj_error_1_2 = ave(proj_error_1_2, n_rep)       
    proj_error_1_3 = proj_error_1_3/n_rep
    for i in range(S):
        exhi_1[i] = error_1_2[i]
        exhi_1[i] = np.vstack((exhi_1[i], proj_error_1_2[i]))
        exhi_1[i] = np.vstack((exhi_1[i], prediction_mse_1_2[i]))
        exhi_1[i] = np.vstack((np.array([[index_array[i]]]), exhi_1[i]))
    print("Obj1 Method 2")
    exhi(exhi_1)
    print("Obj1 Method 3")
    print(error_1_3)
    print(proj_error_1_3)
    print(prediction_mse_1_3)
    
    for i in range(P+1):
        print(f"{i}")
        print(check_stationarity(ar_series[i]))

In [109]:
ar_coeff = [[0.3, 0.2, 0.5, 0.6], [0.5, 0.45, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 5; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.011744 & 0.00031  & 0.010054 \\ \hline
 1 & 0.026797 & 0.001039 & 0.010144 \\ \hline
 2 & 0.026136 & 0.001012 & 0.010138 \\ \hline
 3 & 0.026403 & 0.001033 & 0.010141 \\ \hline
 4 & 0.026445 & 0.001037 & 0.010141 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.01174415]
0.00030994209179671347
0.010054443267347746


In [141]:
ar_coeff = [[0.2, 0.5, 0.2, 0.3], [0.5, 0.4, 0.1, 0.2]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 5; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.051681 & 0.003714 & 0.010541 \\ \hline
 1 & 0.149577 & 0.020258 & 0.012852 \\ \hline
 2 & 0.137057 & 0.01755  & 0.012483 \\ \hline
 3 & 0.138933 & 0.017984 & 0.012544 \\ \hline
 4 & 0.138299 & 0.017857 & 0.012527 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.05168068]
0.0037139562525194325
0.010541288779240026
0
ADF Statistic: -9.71092415154106
p-value: 1.0121101652737118e-16
True
1
ADF Statistic: -6.562669006857199
p-value: 8.305978367952278e-09
True
2
ADF Statistic: -8.817294784125265
p-value: 1.918933292181335e-14
True
3
ADF Statistic: -9.856595927431847
p-value: 4.3429363519643e-17
True
4
ADF Statistic: -6.776118667272482
p-value: 2.5682108395459347e-09
True


In [158]:
ar_coeff = [[0.2, 0.5, 0.01, 0.01], [0.3, 0.4, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.074296 & 0.006734 & 0.010665 \\ \hline
 1 & 0.053839 & 0.004736 & 0.010429 \\ \hline
 2 & 0.034327 & 0.002969 & 0.010236 \\ \hline
 3 & 0.030867 & 0.002688 & 0.010204 \\ \hline
 4 & 0.029905 & 0.002613 & 0.010196 \\ \hline
 5 & 0.029681 & 0.002595 & 0.010194 \\ \hline
 6 & 0.029618 & 0.00259  & 0.010193 \\ \hline
 7 & 0.029602 & 0.002589 & 0.010193 \\ \hline
 8 & 0.029598 & 0.002589 & 0.010193 \\ \hline
 9 & 0.029596 & 0.002589 & 0.010193 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.07429583]
0.006734304653500081
0.010664821373164267
0
ADF Statistic: -13.61488857115552
p-value: 1.843164175175203e-25
True
1
ADF Statistic: -5.998008338390461
p-value: 1.6834352205095565e-07
True
2
ADF Statistic: -33.113847869447504
p-value: 0.0
True
3
ADF Statistic: -31.561145499289754
p-value: 0.0
True
4
ADF Statistic: -6.625953255020538
p-value: 5.875836069046192e-09
True


In [159]:
ar_coeff = [[0.3, 0.6, 0.01, 0.01], [0.3, 0.4, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 12.2288 & 1.17116 & 5.65892 \\ \hline
 1 & 11.9151 & 1.09082 & 5.40342 \\ \hline
 2 & 11.1878 & 1.0538  & 5.20432 \\ \hline
 3 & 10.9054 & 1.03868 & 5.12071 \\ \hline
 4 & 10.7943 & 1.03235 & 5.08625 \\ \hline
 5 & 10.7515 & 1.03009 & 5.07304 \\ \hline
 6 & 10.7354 & 1.02927 & 5.06821 \\ \hline
 7 & 10.7293 & 1.02895 & 5.06634 \\ \hline
 8 & 10.7271 & 1.02884 & 5.06567 \\ \hline
 9 & 10.7263 & 1.02879 & 5.06542 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[12.22882894]
1.1711572501368135
5.658923345520802
0
ADF Statistic: -12.92968701478002
p-value: 3.7232352206934534e-24
True
1
ADF Statistic: -1.2901612364280708
p-value: 0.6336614529626889
False
2
ADF Statistic: -31.191798872159787
p-value: 0.0
True
3
ADF Statistic: -21.080273576864354
p-value: 0.0
True
4
ADF Statistic: -1.6221561455016182
p-value: 0.4715990950653753
False


In [164]:
ar_coeff = [[0.3, 0.6, 0.01, 0.01], [0.3, 0.4, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 500; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 12.4747 & 1.26152 & 3.6372  \\ \hline
 1 & 11.5906 & 1.12582 & 3.40724 \\ \hline
 2 & 10.7365 & 1.04263 & 3.26674 \\ \hline
 3 & 10.43   & 1.01841 & 3.21388 \\ \hline
 4 & 10.3042 & 1.00896 & 3.19189 \\ \hline
 5 & 10.2534 & 1.00524 & 3.18312 \\ \hline
 6 & 10.2339 & 1.00385 & 3.17987 \\ \hline
 7 & 10.2271 & 1.00335 & 3.17869 \\ \hline
 8 & 10.2247 & 1.00317 & 3.17826 \\ \hline
 9 & 10.2238 & 1.0031  & 3.1781  \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[12.47472078]
1.2615223708496073
3.6371999491683504
0
ADF Statistic: -8.18778385599414
p-value: 7.814645547765297e-13
True
1
ADF Statistic: -0.26476682892874176
p-value: 0.9303482928448235
False
2
ADF Statistic: -14.481779269615002
p-value: 6.355087420639318e-27
True
3
ADF Statistic: -22.184278653365336
p-value: 0.0
True
4
ADF Statistic: -0.5623419071670243
p-value: 0.879266220117867
False


In [165]:
ar_coeff = [[0.2, 0.3, 0.01, 0.01], [0.3, 0.4, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.013628 & 0.000627 & 0.010095 \\ \hline
 1 & 0.018272 & 0.001211 & 0.01013  \\ \hline
 2 & 0.018803 & 0.001237 & 0.010132 \\ \hline
 3 & 0.018911 & 0.001244 & 0.010132 \\ \hline
 4 & 0.018919 & 0.001244 & 0.010132 \\ \hline
 5 & 0.018921 & 0.001244 & 0.010132 \\ \hline
 6 & 0.018921 & 0.001244 & 0.010132 \\ \hline
 7 & 0.018921 & 0.001244 & 0.010132 \\ \hline
 8 & 0.018921 & 0.001244 & 0.010132 \\ \hline
 9 & 0.018921 & 0.001244 & 0.010132 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.01362849]
0.0006270519077294461
0.010094724180164629
0
ADF Statistic: -14.504212563958811
p-value: 5.8651005684605185e-27
True
1
ADF Statistic: -9.499891160890916
p-value: 3.467863628948745e-16
True
2
ADF Statistic: -32.92800632030438
p-value: 0.0
True
3
ADF Statistic: -31.27291338363201
p-value: 0.0
True
4
ADF Statistic: -10.806488198183029
p-value: 1.957334056759937e-19
True


In [166]:
ar_coeff = [[0.25, 0.35, 0.01, 0.01], [0.3, 0.4, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.016736 & 0.000751 & 0.00987  \\ \hline
 1 & 0.01747  & 0.00136  & 0.009912 \\ \hline
 2 & 0.018622 & 0.001427 & 0.009922 \\ \hline
 3 & 0.018896 & 0.001439 & 0.009923 \\ \hline
 4 & 0.018909 & 0.00144  & 0.009923 \\ \hline
 5 & 0.018909 & 0.00144  & 0.009923 \\ \hline
 6 & 0.018908 & 0.00144  & 0.009923 \\ \hline
 7 & 0.018908 & 0.00144  & 0.009923 \\ \hline
 8 & 0.018908 & 0.00144  & 0.009923 \\ \hline
 9 & 0.018908 & 0.00144  & 0.009923 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.0167362]
0.0007507109026570622
0.009870150853354876
0
ADF Statistic: -12.31933569691986
p-value: 6.807028100715552e-23
True
1
ADF Statistic: -9.355029526671906
p-value: 8.102594065898628e-16
True
2
ADF Statistic: -31.467494316887695
p-value: 0.0
True
3
ADF Statistic: -21.49078583918745
p-value: 0.0
True
4
ADF Statistic: -10.9798966066448
p-value: 7.504663613569457e-20
True


In [167]:
ar_coeff = [[0.25, 0.35, 0.01, 0.01], [0.35, 0.45, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.019428 & 0.000965 & 0.010163 \\ \hline
 1 & 0.015898 & 0.001134 & 0.010186 \\ \hline
 2 & 0.015539 & 0.001106 & 0.010182 \\ \hline
 3 & 0.015638 & 0.00111  & 0.010182 \\ \hline
 4 & 0.015647 & 0.00111  & 0.010182 \\ \hline
 5 & 0.015648 & 0.00111  & 0.010182 \\ \hline
 6 & 0.015649 & 0.00111  & 0.010183 \\ \hline
 7 & 0.015649 & 0.00111  & 0.010183 \\ \hline
 8 & 0.015649 & 0.00111  & 0.010183 \\ \hline
 9 & 0.015649 & 0.00111  & 0.010183 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.01942808]
0.0009653603813862375
0.010163249578904989
0
ADF Statistic: -12.174464646472694
p-value: 1.3965080198072585e-22
True
1
ADF Statistic: -8.436365943938716
p-value: 1.8121464300847759e-13
True
2
ADF Statistic: -23.72767887358631
p-value: 0.0
True
3
ADF Statistic: -31.8452165416224
p-value: 0.0
True
4
ADF Statistic: -8.343800851870474
p-value: 3.1245082456150857e-13
True


In [154]:
ar_coeff = [[0.3, 0.5, 0.01, 0.01], [0.3, 0.4, 0.01, 0.01]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 500;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.058486 & 0.004441 & 0.010316 \\ \hline
 1 & 0.026588 & 0.002426 & 0.010073 \\ \hline
 2 & 0.018752 & 0.001878 & 0.010016 \\ \hline
 3 & 0.018602 & 0.001846 & 0.010013 \\ \hline
 4 & 0.018704 & 0.001845 & 0.010013 \\ \hline
 5 & 0.018743 & 0.001845 & 0.010013 \\ \hline
 6 & 0.018755 & 0.001845 & 0.010013 \\ \hline
 7 & 0.018758 & 0.001845 & 0.010013 \\ \hline
 8 & 0.018759 & 0.001845 & 0.010013 \\ \hline
 9 & 0.018759 & 0.001845 & 0.010013 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.05848602]
0.004440926831582975
0.010315970029641713
0
ADF Statistic: -4.962704951185172
p-value: 2.6290890263928784e-05
True
1
ADF Statistic: -6.507428492631326
p-value: 1.1220665693580062e-08
True
2
ADF Statistic: -14.871588717053648
p-value: 1.6602029028812167e-27
True
3
ADF Statistic: -21.578481876611068
p-value: 0.0
True
4
ADF Statistic: -6.968431009628373
p-value: 8.791319765549042e-10
True


In [129]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 1;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.011318 & 0.000268 & 0.009706 \\ \hline
 1 & 0.002866 & 0.000267 & 0.009768 \\ \hline
 2 & 0.023486 & 0.000662 & 0.009749 \\ \hline
 3 & 0.036853 & 0.001378 & 0.009831 \\ \hline
 4 & 0.049359 & 0.002332 & 0.009944 \\ \hline
 5 & 0.056546 & 0.00301  & 0.01002  \\ \hline
 6 & 0.062051 & 0.003588 & 0.010087 \\ \hline
 7 & 0.065022 & 0.003917 & 0.010128 \\ \hline
 8 & 0.066359 & 0.004071 & 0.010146 \\ \hline
 9 & 0.066882 & 0.004131 & 0.010154 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.01131842]
0.0002679455523539414
0.009706318228604204
0
ADF Statistic: -10.018989150538435
p-value: 1.698752500405283e-17
True
1
ADF Statistic: -8.986301741851626
p-value: 7.085903873739003e-15
True
2
ADF Statistic: -5.715007802155122
p-value: 7.154438427549727e-07
True
3
ADF Statistic: -6.744854956244183
p-value: 3.0533228672510126e-09
True
4
ADF Statistic: -9.297557167686364
p-value: 1.1353083050610016e-15
True


In [142]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 500;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.008875 & 0.000275 & 0.010087 \\ \hline
 1 & 0.050608 & 0.003034 & 0.010536 \\ \hline
 2 & 0.084375 & 0.007192 & 0.011204 \\ \hline
 3 & 0.108016 & 0.011107 & 0.011833 \\ \hline
 4 & 0.123199 & 0.014034 & 0.012301 \\ \hline
 5 & 0.132316 & 0.015917 & 0.0126   \\ \hline
 6 & 0.137415 & 0.017011 & 0.012775 \\ \hline
 7 & 0.140049 & 0.017586 & 0.012867 \\ \hline
 8 & 0.141417 & 0.017887 & 0.012915 \\ \hline
 9 & 0.142111 & 0.01804  & 0.012939 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.00887459]
0.0002750496001124813
0.010086833060543537
0
ADF Statistic: -8.782694274512012
p-value: 2.3532443047446068e-14
True
1
ADF Statistic: -7.2618269584386725
p-value: 1.6737792259231948e-10
True
2
ADF Statistic: -5.688088485655151
p-value: 8.190554126621876e-07
True
3
ADF Statistic: -5.253300761118566
p-value: 6.836707458053882e-06
True
4
ADF Statistic: -6.742909825704789
p-value: 3.086331575927344e-09
True


In [136]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 2000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.005467 & 9.5e-05  & 0.009872 \\ \hline
 1 & 0.040577 & 0.001796 & 0.010189 \\ \hline
 2 & 0.066103 & 0.004293 & 0.010614 \\ \hline
 3 & 0.082829 & 0.006539 & 0.010992 \\ \hline
 4 & 0.093376 & 0.008145 & 0.011258 \\ \hline
 5 & 0.099728 & 0.00919  & 0.011431 \\ \hline
 6 & 0.103586 & 0.009849 & 0.011539 \\ \hline
 7 & 0.105804 & 0.010235 & 0.011603 \\ \hline
 8 & 0.107025 & 0.010449 & 0.011638 \\ \hline
 9 & 0.107617 & 0.010554 & 0.011655 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.00546669]
9.541168918555044e-05
0.009871546308262725
0
ADF Statistic: -14.406670178360285
p-value: 8.335365340325988e-27
True
1
ADF Statistic: -13.78803712279489
p-value: 9.034830169977841e-26
True
2
ADF Statistic: -8.070329583538978
p-value: 1.555785462674028e-12
True
3
ADF Statistic: -9.637407003233808
p-value: 1.5532219191371885e-16
True
4
ADF Statistic: -14.454290533339785
p-value: 7.015089465177361e-27
True


In [147]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 5; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.020347 & 0.001373 & 0.010187 \\ \hline
 1 & 0.037467 & 0.002894 & 0.010415 \\ \hline
 2 & 0.054417 & 0.004746 & 0.010705 \\ \hline
 3 & 0.066517 & 0.006334 & 0.010943 \\ \hline
 4 & 0.073042 & 0.00726  & 0.011085 \\ \hline
 5 & 0.07677  & 0.007814 & 0.01117  \\ \hline
 6 & 0.07861  & 0.008091 & 0.011211 \\ \hline
 7 & 0.079578 & 0.008237 & 0.011233 \\ \hline
 8 & 0.079996 & 0.008301 & 0.011243 \\ \hline
 9 & 0.080207 & 0.008331 & 0.011247 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.02034651]
0.0013725601260750975
0.01018736508943901
0
ADF Statistic: -9.804612131080384
p-value: 5.871215554779925e-17
True
1
ADF Statistic: -8.592370864243811
p-value: 7.228066743483939e-14
True
2
ADF Statistic: -6.198533580906149
p-value: 5.8840366239061113e-08
True
3
ADF Statistic: -7.36918398880924
p-value: 9.066443109685473e-11
True
4
ADF Statistic: -8.958929219894864
p-value: 8.326182764159356e-15
True


In [146]:
ar_coeff = [[0.7, 0.6, 0.5, 0.6], [0.1, 0.2, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 500; H = 5; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.02609  & 0.002308 & 0.010513 \\ \hline
 1 & 0.042616 & 0.004689 & 0.010903 \\ \hline
 2 & 0.061876 & 0.007113 & 0.011271 \\ \hline
 3 & 0.075181 & 0.009148 & 0.01158  \\ \hline
 4 & 0.081959 & 0.010236 & 0.011741 \\ \hline
 5 & 0.085605 & 0.010821 & 0.011828 \\ \hline
 6 & 0.087514 & 0.01112  & 0.011872 \\ \hline
 7 & 0.088561 & 0.011283 & 0.011897 \\ \hline
 8 & 0.089032 & 0.011353 & 0.011907 \\ \hline
 9 & 0.089291 & 0.011391 & 0.011912 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.02608965]
0.0023078093058364686
0.010512618090558217
0
ADF Statistic: -7.295488592446139
p-value: 1.3815134810483983e-10
True
1
ADF Statistic: -6.438489039155308
p-value: 1.6302328514942316e-08
True
2
ADF Statistic: -4.317676627962533
p-value: 0.0004139235721223278
True
3
ADF Statistic: -4.8701504189600255
p-value: 3.985294801693339e-05
True
4
ADF Statistic: -6.667032168358657
p-value: 4.689423368354186e-09
True


In [132]:
ar_coeff = [[0.3, 0.2, 0.5, 0.6], [0.5, 0.45, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 5; S = 5; n_rep = 1;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.003478 & 1.3e-05  & 0.010025 \\ \hline
 1 & 0.030674 & 0.000931 & 0.010056 \\ \hline
 2 & 0.029539 & 0.000862 & 0.010052 \\ \hline
 3 & 0.029407 & 0.000855 & 0.01005  \\ \hline
 4 & 0.02941  & 0.000855 & 0.01005  \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.00347778]
1.282975244347604e-05
0.01002503592592138
0
ADF Statistic: -8.33498415624334
p-value: 3.29081434060191e-13
True
1
ADF Statistic: -8.754592336677064
p-value: 2.777373342057975e-14
True
2
ADF Statistic: -7.280286288385459
p-value: 1.506645589689754e-10
True
3
ADF Statistic: -6.444471766723444
p-value: 1.5783608632627658e-08
True
4
ADF Statistic: -10.66972593830038
p-value: 4.197367302558998e-19
True


In [133]:
ar_coeff = [[0.3, 0.2, 0.5, 0.6], [0.5, 0.45, 0.4, 0.3]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_burn = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100;
TestAR3(ar_coeff, a1, a2, initial_value, n_burn, n_obs, H, S, n_rep)

Obj1 Method 2
\begin{tabular}{rrrr}
\hline
 0 & 0.012541 & 0.000384 & 0.010086 \\ \hline
 1 & 0.026405 & 0.001045 & 0.010173 \\ \hline
 2 & 0.025706 & 0.001011 & 0.01017  \\ \hline
 3 & 0.025958 & 0.001029 & 0.010172 \\ \hline
 4 & 0.025988 & 0.001032 & 0.010173 \\ \hline
 5 & 0.025995 & 0.001033 & 0.010173 \\ \hline
 6 & 0.025996 & 0.001033 & 0.010173 \\ \hline
 7 & 0.025996 & 0.001033 & 0.010173 \\ \hline
 8 & 0.025996 & 0.001033 & 0.010173 \\ \hline
 9 & 0.025996 & 0.001033 & 0.010173 \\ \hline
\hline \hline
\end{tabular} \hline
Obj1 Method 3
[0.01254139]
0.0003837790253176948
0.010085606296173506
0
ADF Statistic: -8.771071688962135
p-value: 2.520180190479432e-14
True
1
ADF Statistic: -7.9746944225576675
p-value: 2.7221160621279564e-12
True
2
ADF Statistic: -5.286151117031028
p-value: 5.849171528723386e-06
True
3
ADF Statistic: -6.429534887883107
p-value: 1.7110200452098555e-08
True
4
ADF Statistic: -7.772066256334092
p-value: 8.866723634409476e-12
True


In [98]:
compute_eigen(Mhat @ V1[0] @ Mhat, P1, K)

array([[ 0.51731644],
       [ 0.85567701],
       [-0.00747533],
       [-0.0120282 ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [99]:
edr_est_3

array([[ 0.5598016 ],
       [ 0.82840865],
       [ 0.01646763],
       [-0.00949118]])

In [83]:
Mhat @ V1[0] @ Mhat

array([[2286.79517586, 2708.37648782,    8.14821842,  -85.29883119,
           0.        ,    0.        ,    0.        ,    0.        ],
       [2708.37648782, 5281.9686009 ,   11.70324867,   53.72905945,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   8.14821842,   11.70324867,  269.17491866,   33.8127306 ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [ -85.29883119,   53.72905945,   33.8127306 ,  244.67119672,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ],
       [   0.        ,    0.        ,    

In [87]:
np.linalg.inv(np.cov(X3.T)) @ M1 @ np.linalg.inv(np.cov(X3.T))

array([[2286.79517586, 2708.37648782,    8.14821842,  -85.29883119],
       [2708.37648782, 5281.9686009 ,   11.70324867,   53.72905945],
       [   8.14821842,   11.70324867,  269.17491866,   33.8127306 ],
       [ -85.29883119,   53.72905945,   33.8127306 ,  244.67119672]])

In [100]:
compute_eigen(np.linalg.inv(np.cov(X3.T)) @ M1 @ np.linalg.inv(np.cov(X3.T)), P, K)

array([[ 0.51731644],
       [ 0.85567701],
       [-0.00747533],
       [-0.0120282 ]])

In [72]:
np.linalg.inv(np.cov(X3.T))

array([[41.16490901, -9.89853288,  2.59821742, -3.49824055],
       [-9.89853288, 75.17472664, -2.724072  ,  1.7954255 ],
       [ 2.59821742, -2.724072  , 25.5333908 , -0.14301766],
       [-3.49824055,  1.7954255 , -0.14301766, 22.78737646]])

In [73]:
Mhat

array([[41.16490901, -9.89853288,  2.59821742, -3.49824055,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-9.89853288, 75.17472664, -2.724072  ,  1.7954255 ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 2.59821742, -2.724072  , 25.5333908 , -0.14301766,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-3.49824055,  1.7954255 , -0.14301766, 22.78737646,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])

In [77]:
edr_est

array([[ 0.51419669],
       [ 0.85766146],
       [ 0.0012241 ],
       [-0.00413351],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ]])

In [76]:
12.69491037/59.37391691

0.21381291702959673

In [75]:
edr_est

array([[ 0.51419669],
       [ 0.85766146],
       [ 0.0012241 ],
       [-0.00413351],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ]])

In [6]:
X2.shape

(1000, 8)

In [8]:
X3.shape

(1000, 4)

In [80]:
edr_est_2, M = sir_11(X2, y[0], H, K)
compute_eigen(M, P1, K)
# edr_est_2

array([[ 0.79564911],
       [ 0.59559885],
       [-0.00800483],
       [ 0.11018361],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [81]:
edr_est_2

array([[ 0.79564911],
       [ 0.59559885],
       [-0.00800483],
       [ 0.11018361],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [52]:
edr_est_3, _ = sir_11(X3, y[0], H, K)
edr_est_3

array([[-0.71633185],
       [-0.69281519],
       [ 0.00844189],
       [-0.08248953]])

In [5]:
#6. Stationarity Test. You can use the Augmented Dickey-Fuller test to check if the time series is stationary (indicating convergence).
from statsmodels.tsa.stattools import adfuller
def check_stationarity(post_burn_samples):
    result = adfuller(post_burn_samples)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    return result[1] < 0.05  # Stationary if p-value < 0.05
# Example usage:
# check_stationarity(samples, burn_in=1000)
for i in range(P+1):
    print(f"{i}: {ar_series[i]}")
    print(check_stationarity(ar_series[i]))

0: [ 0.17378347 -0.12371793  0.07634439 ... -0.22366648  0.00068112
 -0.02805279]
ADF Statistic: -8.55313458149805
p-value: 9.108683605001782e-14
True
1: [-0.02337402 -0.16199375 -0.10419813 ... -0.15886319 -0.10491914
 -0.15396358]
ADF Statistic: -4.959811870735999
p-value: 2.6637579223943907e-05
True
2: [ 0.1209778   0.16833169  0.22457688 ... -0.40238567 -0.30292872
 -0.40199478]
ADF Statistic: -5.678645416482212
p-value: 8.587638325787732e-07
True
3: [-0.25017749 -0.35220506 -0.34134824 ... -0.02347659  0.1018719
  0.10096104]
ADF Statistic: -6.673130233035014
p-value: 4.534760220298469e-09
True
4: [ 0.          0.         -0.08522755 ... -0.9828543  -0.31626927
 -0.46045895]
ADF Statistic: -7.032641101913379
p-value: 6.128980776411583e-10
True


## True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])

In [5]:
# X_{t}, X_{t-1}

# def TestAR21(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep):
ar_coeff = [[0.1, 0.3, 0.2, 0.2], [0.2, 0.3, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 100
ARnum = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; p = 2; P1 = 2*P; AR = initial_value
noise1 = np.zeros((num_N, n_gene))
noise2 = np.zeros((num_N, n_obs+S+3))
y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]

prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
M = [np.zeros((P, 1)) for _ in range(S)] 
proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]

proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
# error_1_1 is for dim 2p, error_1_2 is for dim p
error_1_1 = [np.zeros((1, )) for _ in range(S)]
error_1_2 = [np.zeros((1, )) for _ in range(S)]

error_2 = [np.zeros((1, )) for _ in range(S)]
g = np.zeros((S, 1))
g1 = np.zeros((S, 1))
g2 = np.zeros((S, 1))
projection_1_norm = np.zeros((S, 1))
projection_2_norm = np.zeros((S, 1))
# True_projection for 2p and p
# True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))

obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
obj2_ave = np.zeros((P1, 1))

error_1_3 = 0
prediction_mse_1_3 = 0
proj_error_1_3 = 0
exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]

# Data Generation
for h in range(num_N):
    noise1[h] = np.random.normal(0, 1, size=(n_gene)) 
    noise2[h] = np.random.normal(0, 1, size=(n_obs+S+3))
    # noise[h] = np.random.normal(0, 0.5, size=(n_obs+S))
ar_series = np.zeros((num_N, n_obs+S+3))
# change initial value and burn-off?
############################################################### 
for t in range(1, n_gene + 1):
    for i in range(4):
        AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
        AR[i][1] = AR[i][0]
        AR[i][0] = AR_temp
for i in range(4):
    for k in range(p):
        ar_series[i][k] = AR[i][k]
for t in range(2, n_obs+S+3):
    for i in range(4):
        ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
    ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]

for a in range(2, S+2):
    y[a-2] = ar_series[4][a:n_obs+a]
    X1[a-2] = np.concatenate((np.array([ar_series[i][a:n_obs+a] for i in range(P)]), np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)])), axis = 0).T #.reshape(-1,1)
# X = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.array([ar_series[i][1:n_obs+1] for i in range(P)])), axis = 0).T
################################################################
# original data generation
# for t in range(0, n_obs+S):
#     ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
#     ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
#     ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
#     ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
#     ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
# for a in range(0, S):
#     y[a] = ar_series[4][a:n_obs+a]
#     X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1, 1) for i in range(P)], axis = 1)
X = X1[0]
V1 = []
for a in range(0, S):
    _, M = sir_11(X, y[a], H, K)
    V1.append(M)
# objective 1: experiment
X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
for q in range(0, S):
    phi = ar_coeff[0] + ar_coeff[1]
    Q3 = np.zeros((P1, P1))
    for j in range(P1):
        for k in range(P1):
            Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T)) @ V1[a] @ np.linalg.inv(np.cov(X.T)))[j, k] * (phi[k] ** a) for a in range(0, q + 1))
    
    # Method 1: dimension 2p 
    edr_est = compute_eigen(Q3, P1, K)
    if edr_est[0] < 0:
        edr_est = -edr_est
    # if edr_est[0] < 0:
    #     edr_est[0:P-1] = -edr_est[0:P-1]
    # if edr_est[P] < 0:
    #     edr_est[P:] = -edr_est[P:]
    # for i in range(ARnum):
    #     edr_est[i*P] = edr_est[i*P]/ar_coeff[i][0]
    #     edr_est[i*P + 1] = edr_est[i*P + 1]/ar_coeff[i][1]
    edr_est = np.real(edr_est / np.linalg.norm(edr_est))
    # error_1[q] += np.mean([abs(edr_est[0] / edr_est[1] - a1/a2), abs(edr_est[P] / edr_est[P + 1] - a1/a2)])
    prediction_mse_1_1[q] += MSE(X @ edr_est, y[0]) 
    proj_error_1_1[q] += np.linalg.norm((proj(edr_est) - proj(True_projection_2p)), 'fro')**2
    
    # Method 2: additional step, dimension p, average every p dimensional vector. 
    # average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
    # error_1_2[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
    # prediction_mse_1_2[q] += MSE((X2 @ average_edr_est).reshape(-1,1), y[0])
    # proj_error_1_2[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2

# Method 3: q = 0 use cross-sectional data X_t y_t, q = 0
# edr_est_3, _ = sir_11(X2, y[0], H, K)
# if edr_est_3[0] < 0:
#     edr_est_3 = -edr_est_3
# edr_est_3 = np.real(edr_est_3 / np.linalg.norm(edr_est_3))
# error_1_3 += abs(edr_est_3[0] / edr_est_3[1] - a1/a2)
# prediction_mse_1_3 += MSE(X2 @ edr_est_3, y[0])
# proj_error_1_3 += np.linalg.norm((proj(edr_est_3) - proj(True_projection_p)), 'fro')**2

In [6]:
# True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
edr_est

array([[ 5.63687861e-01],
       [ 8.25513931e-01],
       [ 2.62686594e-04],
       [ 1.45438402e-03],
       [ 1.52648897e-02],
       [-2.29093176e-02],
       [-4.64505041e-03],
       [-1.06370818e-03]])

In [112]:
#add risk to be averaged over replicas, add prediction error for obj2 for averaged direction
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
def ave(arr, N):
    for i in range(len(arr)):
        arr[i] = arr[i]/N 
    return arr 
def compute_eigen(Q4, P, K):
    eigenvalues2, eigenvectors2 = np.linalg.eig(Q4)
    K_index = np.argpartition(np.abs(eigenvalues2), P - K) >= P - K
    K_largest_eigenvectors = eigenvectors2[:, K_index]
    edr_est = K_largest_eigenvectors  
    if edr_est[0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    return edr_est
    
def proj(edr_est): 
    E = edr_est @ np.linalg.inv(edr_est.T @ edr_est) @ edr_est.T
    return E

def exhi(obj1):        
    array1 = np.vectorize(lambda x: f"{x:.6f}")(obj1)
    table = tabulate(array1, tablefmt='latex_raw')
    # Split the table into lines
    lines = table.split('\n')
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    print(latex_table)
    
def MSE(X, y):
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate errors
    mse = mean_squared_error(y_test, y_pred)
    return mse

def TestAR3(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep):
    ARnum = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; p = 2; P1 = 2*P; AR = initial_value
    noise1 = np.zeros((num_N, n_gene))
    noise2 = np.zeros((num_N, n_obs+S+3))
    y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
    X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
    obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
    
    prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]
    
    prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
    M = [np.zeros((P, 1)) for _ in range(S)] 
    proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
    proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]
    
    proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
    # error_1_1 is for dim 2p, error_1_2 is for dim p
    error_1_1 = [np.zeros((1, )) for _ in range(S)]
    error_1_2 = [np.zeros((1, )) for _ in range(S)]
    
    error_2 = [np.zeros((1, )) for _ in range(S)]
    g = np.zeros((S, 1))
    g1 = np.zeros((S, 1))
    g2 = np.zeros((S, 1))
    projection_1_norm = np.zeros((S, 1))
    projection_2_norm = np.zeros((S, 1))
    # True_projection for 2p and p
    # True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
    True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
    True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))
    
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    obj2_ave = np.zeros((P1, 1))

    error_1_3 = 0
    prediction_mse_1_3 = 0
    proj_error_1_3 = 0
    
    exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    while n1 < n_rep:  
        obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
        obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
        # Data Generation
        for h in range(num_N):
            noise1[h] = np.random.normal(0, 1, size=(n_gene)) 
            noise2[h] = np.random.normal(0, 1, size=(n_obs+S+3))
            # noise[h] = np.random.normal(0, 0.5, size=(n_obs+S))
        ar_series = np.zeros((num_N, n_obs+S+3))
        # change initial value and burn-off?
        ############################################################### 
        for t in range(1, n_gene + 1):
            for i in range(4):
                AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
                AR[i][1] = AR[i][0]
                AR[i][0] = AR_temp
        for i in range(4):
            for k in range(p):
                ar_series[i][k] = AR[i][k]
        for t in range(2, n_obs+S+3):
            for i in range(4):
                ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]
        
        for a in range(2, S+2):
            y[a-2] = ar_series[4][a:n_obs+a]
            X1[a-2] = np.concatenate((np.array([ar_series[i][a:n_obs+a] for i in range(P)]), np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)])), axis = 0).T #.reshape(-1,1)
        # X = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.array([ar_series[i][1:n_obs+1] for i in range(P)])), axis = 0).T
        ################################################################
        # original data generation
        # for t in range(0, n_obs+S):
        #     ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
        #     ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
        #     ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
        #     ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
        #     ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        # for a in range(0, S):
        #     y[a] = ar_series[4][a:n_obs+a]
        #     X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1, 1) for i in range(P)], axis = 1)
        X = X1[0]
        V1 = []
        for a in range(0, S):
            _, M = sir_11(X, y[a], H, K)
            V1.append(M)
    # objective 1: experiment
        X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
        for q in range(0, S):
            phi = ar_coeff[0] + ar_coeff[1]
            Q3 = np.zeros((P1, P1))
            for j in range(P1):
                for k in range(P1):
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T)) @ V1[a] @ np.linalg.inv(np.cov(X.T)))[j, k] * (phi[k] ** a) for a in range(0, q + 1))
            
            # Method 1: dimension 2p 
            edr_est = compute_eigen(Q3, P1, K)
            if edr_est[0] < 0:
                edr_est = -edr_est
            # for i in range(ARnum):
            #     edr_est[i*P] = edr_est[i*P]/ar_coeff[i][0]
            #     edr_est[i*P + 1] = edr_est[i*P + 1]/ar_coeff[i][1]
            edr_est = np.real(edr_est / np.linalg.norm(edr_est))
            error_1_1[q] += abs(edr_est[0] / edr_est[1] - a1/a2)
            prediction_mse_1_1[q] += MSE(X @ edr_est, y[0]) 
            proj_error_1_1[q] += np.linalg.norm((proj(edr_est) - proj(True_projection_2p)), 'fro')**2
            
            # Method 2: additional step, dimension p, average every p dimensional vector. 
            # average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
            # error_1_2[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
            # prediction_mse_1_2[q] += MSE((X2 @ average_edr_est).reshape(-1,1), y[0])
            # proj_error_1_2[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
        
        # Method 3: q = 0 use cross-sectional data X_t y_t, q = 0
        # edr_est_3, _ = sir_11(X2, y[0], H, K)
        # if edr_est_3[0] < 0:
        #     edr_est_3 = -edr_est_3
        # edr_est_3 = np.real(edr_est_3 / np.linalg.norm(edr_est_3))
        # error_1_3 += abs(edr_est_3[0] / edr_est_3[1] - a1/a2)
        # prediction_mse_1_3 += MSE(X2 @ edr_est_3, y[0])
        # proj_error_1_3 += np.linalg.norm((proj(edr_est_3) - proj(True_projection_p)), 'fro')**2
                
        n1 += 1  
        
    error_1_1 = ave(error_1_1, n_rep)    
    error_1_2 = ave(error_1_2, n_rep)
    error_1_3 = error_1_3/n_rep
    prediction_mse_1_1 = ave(prediction_mse_1_1, n_rep)
    prediction_mse_1_2 = ave(prediction_mse_1_2, n_rep)
    prediction_mse_1_3 = prediction_mse_1_3/n_rep
    proj_error_1_1 = ave(proj_error_1_1, n_rep)
    proj_error_1_2 = ave(proj_error_1_2, n_rep)       
    proj_error_1_3 = proj_error_1_3/n_rep
    
    index_array = list(range(len(obj_1)))
    for i in range(S):
        exhi_1[i] = error_1_1[i]
        exhi_1[i] = np.vstack((exhi_1[i], proj_error_1_1[i]))
        exhi_1[i] = np.vstack((exhi_1[i], prediction_mse_1_1[i]))
        exhi_1[i] = np.vstack((np.array([[index_array[i]]]), exhi_1[i]))
    
    # for i in range(S):
    #     exhi_2[i] = error_1_2[i]
    #     exhi_2[i] = np.vstack((exhi_2[i], proj_error_1_2[i]))
    #     exhi_2[i] = np.vstack((exhi_2[i], prediction_mse_1_2[i]))
    #     exhi_2[i] = np.vstack((np.array([[index_array[i]]]), exhi_2[i]))
    
    #     obj_1[i] = np.vstack((obj_1[i], g[i].reshape(1,-1)))
    #     obj_1[i] = np.vstack((np.array([[index_array[i]]]), obj_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), proj_error_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), mse[i]))
      
    #objective 1: exhibition
    # for i in range(S):
    #     g[i] = abs(obj_1[i][0] / obj_1[i][1] - a1/a2)
    #     obj_1[i] = np.vstack((obj_1[i], g[i].reshape(1,-1)))
    #     obj_1[i] = np.vstack((np.array([[index_array[i]]]), obj_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), proj_error_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), mse[i]))
        
    # Result1 = [g[0] - min(g), projection1_norm[0] - min(projection1_norm), mse[0] - min(mse)]
    # Result1 = [x.item() for x in Result1]
        
    #objective 2: exhibition
    
    # for i in range(S):
    #     g[i] = abs(obj2[i][0] / obj2[i][1] - a1/a2)
    #     projection2_norm[i] = np.linalg.norm(proj2[i] - proj(True_projection), 'fro')
    #     obj2[i] = np.vstack((obj_2_avevec[i], g[i].reshape(1,-1)))
    #     obj2[i] = np.vstack((np.array([[index_array[i]]]), obj_2_avevec[i]))    
    #     obj2[i] = np.vstack((np.array(obj2[i]), projection2_norm[i]))
    #     obj2[i] = np.vstack((np.array(obj2[i]), mse2[i]))
    #     obj2[i] = np.vstack((np.array(obj2[i]), abs(obj_2_avevec[i][0]/obj_2_avevec[i][1] - 2/3)))
    #     obj2[i] = np.vstack((np.array(obj2[i]), np.linalg.norm(proj(obj_2_avevec[i]) - proj(True_projection), 'fro')))
    #     obj2[i] = np.vstack((np.array(obj2[i]), Mse1(X @ obj_2_avevec[i], y[0])))
    
    # for i in range(S):
    #     g[i] = abs(obj_2_avevec[i][0] / obj_2_avevec[i][1] - a1/a2)
    #     g1[i] = proj_error_2[i]
    #     g2[i] = Mse1(X @ obj_2_avevec[i], y[0])
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g[i].reshape(1,-1)))
    #     obj_2_avevec[i] = np.vstack((np.array([[index_array[i]]]), obj_2_avevec[i]))    
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g1[i]))
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g2[i]))
    
    # Result2 = [g[0] - min(g), g1[0] - min(g1), g2[0] - min(g2)] 
    # Result2 = [x.item() for x in Result2]
    
    # print("error_1:",[x.item() for x in error_1])
    # print("error_2:",[x.item() for x in error_2])
    # print("proj_error_1:",[x.item() for x in proj_error_1]) 
    # print("proj_error_2:",[x.item() for x in proj_error_2])
    
    # print(obj_2_avevec)
    # print(obj_2)
    # print("Obj1")
    # exhi(obj1) 
    # print("Obj2")
    # exhi(obj_2_avevec)
    # print(Result1), print(Result2)
    # return obj_2_avevec
    print("Obj1 Method 1")
    exhi(exhi_1)
    # print("Obj1 Method 2")
    # exhi(exhi_2)
    # # print("Obj1 Method 3")
    # return error_1_3, proj_error_1_3, prediction_mse_1_3
    return error_1_1

In [113]:
ar_coeff = [[0.1, 0.3, 0.2, 0.2], [0.2, 0.3, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
TestAR3(ar_coeff, 2, 3, initial_value, 10000, 1000, 50, 10, 500)

Obj1 Method 1
\begin{tabular}{rrrr}
\hline
 0 & 0.011961 & 0.002154 & 1.00158  \\ \hline
 1 & 0.013026 & 0.001983 & 0.999956 \\ \hline
 2 & 0.013383 & 0.002    & 0.999882 \\ \hline
 3 & 0.013402 & 0.002004 & 0.999989 \\ \hline
 4 & 0.013403 & 0.002005 & 1.00002  \\ \hline
 5 & 0.013403 & 0.002006 & 1.00005  \\ \hline
 6 & 0.013403 & 0.002006 & 1.00006  \\ \hline
 7 & 0.013403 & 0.002007 & 1.00007  \\ \hline
 8 & 0.013403 & 0.002007 & 1.00008  \\ \hline
 9 & 0.013403 & 0.002007 & 1.00008  \\ \hline
\hline \hline
\end{tabular} \hline


[array([0.01196092]),
 array([0.01302558]),
 array([0.01338339]),
 array([0.0134016]),
 array([0.01340321]),
 array([0.01340333]),
 array([0.01340335]),
 array([0.01340335]),
 array([0.01340335]),
 array([0.01340335])]

In [114]:
ar_coeff = [[0.6, 0.7, 0.3, 0.2], [0.2, 0.3, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
TestAR3(ar_coeff, 2, 3, initial_value, 10000, 1000, 50, 10, 500)

Obj1 Method 1
\begin{tabular}{rrrr}
\hline
 0 & 0.396902 & 0.816756 & 629.061  \\ \hline
 1 & 0.335332 & 0.69138  & 240.623  \\ \hline
 2 & 0.290057 & 0.630316 & 146.835  \\ \hline
 3 & 0.271181 & 0.604609 & 117.789  \\ \hline
 4 & 0.262416 & 0.593056 & 105.571  \\ \hline
 5 & 0.258148 & 0.587598 & 100.191  \\ \hline
 6 & 0.256406 & 0.584916 &  97.7121 \\ \hline
 7 & 0.255456 & 0.583547 &  96.6016 \\ \hline
 8 & 0.254982 & 0.582944 &  96.1311 \\ \hline
 9 & 0.254762 & 0.582664 &  95.932  \\ \hline
\hline \hline
\end{tabular} \hline


[array([0.39690151]),
 array([0.33533157]),
 array([0.29005721]),
 array([0.27118087]),
 array([0.26241614]),
 array([0.25814783]),
 array([0.25640647]),
 array([0.25545577]),
 array([0.2549821]),
 array([0.25476185])]

In [115]:
ar_coeff = [[0.2, 0.3, 0.3, 0.2], [0.2, 0.3, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
TestAR3(ar_coeff, 2, 3, initial_value, 10000, 1000, 50, 10, 500)

Obj1 Method 1
\begin{tabular}{rrrr}
\hline
 0 & 0.011696 & 0.002334 & 1.00995 \\ \hline
 1 & 0.011966 & 0.002052 & 1.00841 \\ \hline
 2 & 0.012132 & 0.002061 & 1.00849 \\ \hline
 3 & 0.012144 & 0.002064 & 1.00864 \\ \hline
 4 & 0.012144 & 0.002066 & 1.00868 \\ \hline
 5 & 0.012144 & 0.002066 & 1.00871 \\ \hline
 6 & 0.012144 & 0.002067 & 1.00873 \\ \hline
 7 & 0.012144 & 0.002067 & 1.00874 \\ \hline
 8 & 0.012144 & 0.002067 & 1.00874 \\ \hline
 9 & 0.012144 & 0.002067 & 1.00875 \\ \hline
\hline \hline
\end{tabular} \hline


[array([0.01169595]),
 array([0.0119663]),
 array([0.01213212]),
 array([0.01214352]),
 array([0.01214441]),
 array([0.01214448]),
 array([0.01214448]),
 array([0.01214447]),
 array([0.01214447]),
 array([0.01214447])]

In [116]:
ar_coeff = [[0.4, 0.5, 0.3, 0.2], [0.4, 0.5, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
TestAR3(ar_coeff, 2, 3, initial_value, 10000, 1000, 50, 10, 500)

Obj1 Method 1
\begin{tabular}{rrrr}
\hline
 0 & 0.308093 & 0.689584 & 402.888 \\ \hline
 1 & 0.248962 & 0.71341  & 420.198 \\ \hline
 2 & 0.233869 & 0.71651  & 422.399 \\ \hline
 3 & 0.23032  & 0.718962 & 424.706 \\ \hline
 4 & 0.229278 & 0.720253 & 426.456 \\ \hline
 5 & 0.22919  & 0.721058 & 427.374 \\ \hline
 6 & 0.22924  & 0.72158  & 427.996 \\ \hline
 7 & 0.229288 & 0.721908 & 428.408 \\ \hline
 8 & 0.229333 & 0.722138 & 428.682 \\ \hline
 9 & 0.229364 & 0.722284 & 428.851 \\ \hline
\hline \hline
\end{tabular} \hline


[array([0.30809279]),
 array([0.24896211]),
 array([0.23386874]),
 array([0.23032022]),
 array([0.22927754]),
 array([0.22919001]),
 array([0.22924048]),
 array([0.22928785]),
 array([0.22933297]),
 array([0.22936371])]

# $X_{t}, X_{t-1}, \text{Objective 1 and objective 2}, AR(p)$

In [ ]:
# objective 2: experiment
for q in range(0, S):
    Q4 = np.linalg.inv(np.cov(X1[q].T)) @ V1[q] @ np.linalg.inv(np.cov(X1[q].T))   # multiply np.linalg.inv(np.cov(X1[q].T)), by stationarity, it should cause no influence.
    # Q3 = np.linalg.inv(np.cov(X.T)) @ V1[q] @ np.linalg.inv(np.cov(X.T)), if we multiply np.linalg.inv(np.cov(X.T)) like this line, result for q = 0 should be the same.   
    K_largest_eigenvectors = compute_eigen(Q4, P1, K)
    edr_est1 = np.multiply(np.power(phi, -q), K_largest_eigenvectors.flatten())   
    if edr_est1[0] < 0:
        edr_est1 = -edr_est1
    edr_est1 = edr_est1 / np.linalg.norm(edr_est1)
    obj2[q] += np.real(edr_est1.reshape(-1,1))

# $X_{t-1}, X_{t-2}$

## True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])

In [84]:
# X_{t-1}, X_{t-2}

ar_coeff = [[0.1, 0.3, 0.2, 0.2], [0.2, 0.3, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
a1 = 2;a2 = 3; n_gene = 10000; n_obs = 1000; H = 50; S = 10; n_rep = 500;
ARnum = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; p = 2; P1 = 2*P; AR = initial_value
noise1 = np.zeros((num_N, n_gene))
noise2 = np.zeros((num_N, n_obs+S+3))
y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]

prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
M = [np.zeros((P, 1)) for _ in range(S)] 
proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]

proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
# error_1_1 is for dim 2p, error_1_2 is for dim p
error_1_1 = [np.zeros((1, )) for _ in range(S)]
error_1_2 = [np.zeros((1, )) for _ in range(S)]

error_2 = [np.zeros((1, )) for _ in range(S)]
g = np.zeros((S, 1))
g1 = np.zeros((S, 1))
g2 = np.zeros((S, 1))
projection_1_norm = np.zeros((S, 1))
projection_2_norm = np.zeros((S, 1))
# True_projection for 2p and p
True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))

obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
obj2_ave = np.zeros((P1, 1))

error_1_3 = 0
prediction_mse_1_3 = 0
proj_error_1_3 = 0
exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
while n1 < n_rep:
    # Data Generation
    for h in range(num_N):
        noise1[h] = np.random.normal(0, 0.1, size=(n_gene)) 
        noise2[h] = np.random.normal(0, 0.1, size=(n_obs+S+3))
        # noise[h] = np.random.normal(0, 0.5, size=(n_obs+S))
    ar_series = np.zeros((num_N, n_obs+S+3))
    # change initial value and burn-off?
    ############################################################### 
    for t in range(1, n_gene + 1):
        for i in range(4):
            AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
            AR[i][1] = AR[i][0]
            AR[i][0] = AR_temp
    for i in range(4):
        for k in range(p):
            ar_series[i][k] = AR[i][k]
    for t in range(2, n_obs+S+3):
        for i in range(4):
            ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
        ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]
    
    for a in range(2, S+2):
        y[a-2] = ar_series[4][a:n_obs+a]
        X1[a-2] = np.concatenate((np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)]), np.array([ar_series[i][a-2:n_obs+a-2] for i in range(P)])), axis = 0).T #.reshape(-1,1)
    # X = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.array([ar_series[i][1:n_obs+1] for i in range(P)])), axis = 0).T
    ################################################################
    # original data generation
    # for t in range(0, n_obs+S):
    #     ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
    #     ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
    #     ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
    #     ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
    #     ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
    # for a in range(0, S):
    #     y[a] = ar_series[4][a:n_obs+a]
    #     X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1, 1) for i in range(P)], axis = 1)
    X = X1[0]
    V1 = []
    for a in range(0, S):
        _, M = sir_11(X, y[a], H, K)
        V1.append(M)
    # objective 1: experiment
    X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
    for q in range(0, S):
        phi = ar_coeff[0] + ar_coeff[1]
        Q3 = np.zeros((P1, P1))
        for j in range(P1):
            for k in range(P1):
                Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T)) @ V1[a] @ np.linalg.inv(np.cov(X.T)))[j, k] * (phi[k] ** a) for a in range(0, q + 1))
        
        # Method 1: dimension 2p 
        edr_est = compute_eigen(Q3, P1, K)
        if edr_est[0] < 0:
            edr_est[0:P] = -edr_est[0:P]
        if edr_est[P] < 0:
            edr_est[P:] = -edr_est[P:]
        for i in range(ARnum):
            edr_est[i*P] = edr_est[i*P]/ar_coeff[i][0]
            edr_est[i*P + 1] = edr_est[i*P + 1]/ar_coeff[i][1]
        edr_est = np.real(edr_est / np.linalg.norm(edr_est))
        # error_1[q] += np.mean([abs(edr_est[0] / edr_est[1] - a1/a2), abs(edr_est[P] / edr_est[P + 1] - a1/a2)])
        prediction_mse_1_1[q] += MSE(X @ edr_est, y[0]) 
        proj_error_1_1[q] += np.linalg.norm((proj(edr_est) - proj(True_projection_2p)), 'fro')**2
        
        # Method 2: additional step, dimension p, average every p dimensional vector. 
        average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
        error_1_2[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
        prediction_mse_1_2[q] += MSE((X2 @ average_edr_est).reshape(-1,1), y[0])
        proj_error_1_2[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
    
    # Method 3: q = 0 use cross-sectional data X_t y_t, q = 0
    edr_est_3, _ = sir_11(X2, y[0], H, K)
    if edr_est_3[0] < 0:
        edr_est_3 = -edr_est_3
    edr_est_3 = np.real(edr_est_3 / np.linalg.norm(edr_est_3))
    error_1_3 += abs(edr_est_3[0] / edr_est_3[1] - a1/a2)
    prediction_mse_1_3 += MSE(X2 @ edr_est_3, y[0])
    proj_error_1_3 += np.linalg.norm((proj(edr_est_3) - proj(True_projection_p)), 'fro')**2

    n1 += 1

error_1_2 = ave(error_1_2, n_rep)
error_1_3 = error_1_3/n_rep

In [85]:
average_edr_est

array([ 0.43848314,  0.52013094,  0.01398793, -0.00101618])

In [86]:
# True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
edr_est

array([[ 0.61553297],
       [ 0.5949419 ],
       [ 0.01607944],
       [ 0.006191  ],
       [ 0.26143331],
       [ 0.44531999],
       [ 0.01189641],
       [-0.00822335]])

In [87]:
error_1_2

[array([5.20692684]),
 array([0.89550178]),
 array([0.87719042]),
 array([0.91310142]),
 array([0.91255364]),
 array([0.91244411]),
 array([0.91286836]),
 array([0.91344171]),
 array([0.91368023]),
 array([0.9139301])]

In [88]:
error_1_3

array([0.28573143])

In [77]:
(ar_coeff[0][0]*edr_est[0] + ar_coeff[0][1]*edr_est[1])/2

array([0.11322715])

In [78]:
(ar_coeff[1][0]*edr_est[4] + ar_coeff[1][1]*edr_est[5])/2

array([0.00243752])

## True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])

In [64]:
#add risk to be averaged over replicas, add prediction error for obj2 for averaged direction
from tabulate import tabulate
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
def ave(arr, N):
    for i in range(len(arr)):
        arr[i] = arr[i]/N 
    return arr 
def compute_eigen(Q4, P, K):
    eigenvalues2, eigenvectors2 = np.linalg.eig(Q4)
    K_index = np.argpartition(np.abs(eigenvalues2), P - K) >= P - K
    K_largest_eigenvectors = eigenvectors2[:, K_index]
    edr_est = K_largest_eigenvectors  
    if edr_est[0] < 0:
        edr_est = -edr_est
    edr_est = edr_est / np.linalg.norm(edr_est)
    return edr_est
    
def proj(edr_est): 
    E = edr_est @ np.linalg.inv(edr_est.T @ edr_est) @ edr_est.T
    return E

def exhi(obj1):        
    array1 = np.vectorize(lambda x: f"{x:.6f}")(obj1)
    table = tabulate(array1, tablefmt='latex_raw')
    # Split the table into lines
    lines = table.split('\n')
    # Insert \hline after each row
    latex_table = '\n'.join([line + (' \\hline' if (idx > 1) else '') for idx, line in enumerate(lines)])
    print(latex_table)
    
def MSE(X, y):
    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Create and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate errors
    mse = mean_squared_error(y_test, y_pred)
    return mse

def TestAR5(ar_coeff, a1, a2, initial_value, n_gene, n_obs, H, S, n_rep):
    ARnum = 2; num_N = 5;P = 4;K = 1;n1 = 0;l = 1; p = 2; P1 = 2*P; AR = initial_value
    noise1 = np.zeros((num_N, n_gene))
    noise2 = np.zeros((num_N, n_obs+S+3))
    y = [np.zeros((num_N, n_obs+i)) for i in range(S)]
    X1 = [[np.zeros((num_N, 2*n_obs)) for i in range(P)] for i in range(S)]
    obj_1 = [np.zeros((P, 1)) for _ in range(S)]  
    obj_2 = [np.zeros((P1, 1)) for _ in range(S)]  
    
    prediction_mse_1_1 = [np.zeros((1, 1)) for _ in range(S)]
    prediction_mse_1_2 = [np.zeros((1, 1)) for _ in range(S)]
    
    prediction_mse_2 = [np.zeros((1, 1)) for _ in range(S)]
    M = [np.zeros((P, 1)) for _ in range(S)] 
    proj_error_1_1 = [np.zeros((1, )) for _ in range(S)] 
    proj_error_1_2 = [np.zeros((1, )) for _ in range(S)]
    
    proj_error_2 = [np.zeros((1, )) for _ in range(S)] 
    # error_1_1 is for dim 2p, error_1_2 is for dim p
    error_1_1 = [np.zeros((1, )) for _ in range(S)]
    error_1_2 = [np.zeros((1, )) for _ in range(S)]
    
    error_2 = [np.zeros((1, )) for _ in range(S)]
    g = np.zeros((S, 1))
    g1 = np.zeros((S, 1))
    g2 = np.zeros((S, 1))
    projection_1_norm = np.zeros((S, 1))
    projection_2_norm = np.zeros((S, 1))
    # True_projection for 2p and p
    # True_projection_2p = np.array([[2],[3],[0],[0],[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[2],[3],[0],[0]]))))
    True_projection_2p = np.array([[2],[3],[0],[0],[0],[0],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0],[0],[0],[0],[0]]))))
    True_projection_p = np.array([[2],[3],[0],[0]])/((np.linalg.norm(np.array([[2],[3],[0],[0]]))))
    
    obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
    obj2_ave = np.zeros((P1, 1))

    error_1_3 = 0
    prediction_mse_1_3 = 0
    proj_error_1_3 = 0
    
    exhi_1 = [np.zeros((P, 1)) for _ in range(S)]
    exhi_2 = [np.zeros((P, 1)) for _ in range(S)]
    while n1 < n_rep:  
        obj_2 = [np.zeros((P1, 1)) for _ in range(S)]
        obj_2_avevec = [np.zeros((P1, 1)) for _ in range(S)]
        # Data Generation
        for h in range(num_N):
            noise1[h] = np.random.normal(0, 1, size=(n_gene)) 
            noise2[h] = np.random.normal(0, 1, size=(n_obs+S+3))
            # noise[h] = np.random.normal(0, 0.5, size=(n_obs+S))
        ar_series = np.zeros((num_N, n_obs+S+3))
        # change initial value and burn-off?
        ############################################################### 
        for t in range(1, n_gene + 1):
            for i in range(4):
                AR_temp = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([AR[i][0], AR[i][1]]) + noise1[i][t-1]
                AR[i][1] = AR[i][0]
                AR[i][0] = AR_temp
        for i in range(4):
            for k in range(p):
                ar_series[i][k] = AR[i][k]
        for t in range(2, n_obs+S+3):
            for i in range(4):
                ar_series[i][t] = np.array([ar_coeff[0][i], ar_coeff[1][i]]) @ np.array([ar_series[i][t - 1], ar_series[i][t - 2]]) + noise2[i][t]
            ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise2[4][t]
        
        for a in range(2, S+2):
            y[a-2] = ar_series[4][a:n_obs+a]
            X1[a-2] = np.concatenate((np.array([ar_series[i][a-1:n_obs+a-1] for i in range(P)]), np.array([ar_series[i][a-2:n_obs+a-2] for i in range(P)])), axis = 0).T #.reshape(-1,1)
        # X = np.concatenate((np.array([ar_series[i][2:n_obs+2] for i in range(P)]), np.array([ar_series[i][1:n_obs+1] for i in range(P)])), axis = 0).T
        ################################################################
        # original data generation
        # for t in range(0, n_obs+S):
        #     ar_series[0][t] = ar_coeff[0] * ar_series[0][t - 1] + noise[0][t]
        #     ar_series[1][t] = ar_coeff[1] * ar_series[1][t - 1] + noise[1][t]
        #     ar_series[2][t] = ar_coeff[2] * ar_series[2][t - 1] + noise[2][t]
        #     ar_series[3][t] = ar_coeff[3] * ar_series[3][t - 1] + noise[3][t]
        #     ar_series[4][t] = a1 * ar_series[0][t] + a2 * ar_series[1][t] + noise[4][t]
        # for a in range(0, S):
        #     y[a] = ar_series[4][a:n_obs+a]
        #     X1[a] = np.concatenate([ar_series[i][a:n_obs+a].reshape(-1, 1) for i in range(P)], axis = 1)
        X = X1[0]
        V1 = []
        for a in range(0, S):
            _, M = sir_11(X, y[a], H, K)
            V1.append(M)
    # objective 1: experiment
        X2 = np.concatenate((np.array([ar_series[i][2:n_obs+2].reshape(n_obs, 1) for i in range(P)])), axis = 1)
        for q in range(0, S):
            phi = ar_coeff[0] + ar_coeff[1]
            Q3 = np.zeros((P1, P1))
            for j in range(P1):
                for k in range(P1):
                    Q3[j, k] = sum((phi[j] ** a) * (np.linalg.inv(np.cov(X.T)) @ V1[a] @ np.linalg.inv(np.cov(X.T)))[j, k] * (phi[k] ** a) for a in range(0, q + 1))
            
            # Method 1: dimension 2p 
            edr_est = compute_eigen(Q3, P1, K)
            if edr_est[0] < 0:
                edr_est = -edr_est
            # for i in range(ARnum):
            #     edr_est[i*P] = edr_est[i*P]/ar_coeff[i][0]
            #     edr_est[i*P + 1] = edr_est[i*P + 1]/ar_coeff[i][1]
            edr_est = np.real(edr_est / np.linalg.norm(edr_est))
            error_1_1[q] += abs(edr_est[0] / edr_est[1] - a1/a2)
            prediction_mse_1_1[q] += MSE(X @ edr_est, y[0]) 
            proj_error_1_1[q] += np.linalg.norm((proj(edr_est) - proj(True_projection_2p)), 'fro')**2
            
            # Method 2: additional step, dimension p, average every p dimensional vector. 
            # average_edr_est = np.mean(edr_est.reshape(-1, P), axis=0)
            # error_1_2[q] += abs(average_edr_est[0] / average_edr_est[1] - a1/a2)
            # prediction_mse_1_2[q] += MSE((X2 @ average_edr_est).reshape(-1,1), y[0])
            # proj_error_1_2[q] += np.linalg.norm((proj((average_edr_est).reshape(-1,1)) - proj(True_projection_p)), 'fro')**2
        
        # Method 3: q = 0 use cross-sectional data X_t y_t, q = 0
        # edr_est_3, _ = sir_11(X2, y[0], H, K)
        # if edr_est_3[0] < 0:
        #     edr_est_3 = -edr_est_3
        # edr_est_3 = np.real(edr_est_3 / np.linalg.norm(edr_est_3))
        # error_1_3 += abs(edr_est_3[0] / edr_est_3[1] - a1/a2)
        # prediction_mse_1_3 += MSE(X2 @ edr_est_3, y[0])
        # proj_error_1_3 += np.linalg.norm((proj(edr_est_3) - proj(True_projection_p)), 'fro')**2
                
        n1 += 1  
        
    error_1_1 = ave(error_1_1, n_rep)    
    error_1_2 = ave(error_1_2, n_rep)
    error_1_3 = error_1_3/n_rep
    prediction_mse_1_1 = ave(prediction_mse_1_1, n_rep)
    prediction_mse_1_2 = ave(prediction_mse_1_2, n_rep)
    prediction_mse_1_3 = prediction_mse_1_3/n_rep
    proj_error_1_1 = ave(proj_error_1_1, n_rep)
    proj_error_1_2 = ave(proj_error_1_2, n_rep)       
    proj_error_1_3 = proj_error_1_3/n_rep
    
    index_array = list(range(len(obj_1)))
    for i in range(S):
        exhi_1[i] = prediction_mse_1_1[i]
        exhi_1[i] = np.vstack((exhi_1[i], proj_error_1_1[i]))
        exhi_1[i] = np.vstack((np.array([[index_array[i]]]), exhi_1[i]))
    
    # for i in range(S):
    #     exhi_2[i] = error_1_2[i]
    #     exhi_2[i] = np.vstack((exhi_2[i], proj_error_1_2[i]))
    #     exhi_2[i] = np.vstack((exhi_2[i], prediction_mse_1_2[i]))
    #     exhi_2[i] = np.vstack((np.array([[index_array[i]]]), exhi_2[i]))
    
    #     obj_1[i] = np.vstack((obj_1[i], g[i].reshape(1,-1)))
    #     obj_1[i] = np.vstack((np.array([[index_array[i]]]), obj_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), proj_error_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), mse[i]))
      
    #objective 1: exhibition
    # for i in range(S):
    #     g[i] = abs(obj_1[i][0] / obj_1[i][1] - a1/a2)
    #     obj_1[i] = np.vstack((obj_1[i], g[i].reshape(1,-1)))
    #     obj_1[i] = np.vstack((np.array([[index_array[i]]]), obj_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), proj_error_1[i])) 
    #     obj_1[i] = np.vstack((np.array(obj_1[i]), mse[i]))
        
    # Result1 = [g[0] - min(g), projection1_norm[0] - min(projection1_norm), mse[0] - min(mse)]
    # Result1 = [x.item() for x in Result1]
        
    #objective 2: exhibition
    
    # for i in range(S):
    #     g[i] = abs(obj2[i][0] / obj2[i][1] - a1/a2)
    #     projection2_norm[i] = np.linalg.norm(proj2[i] - proj(True_projection), 'fro')
    #     obj2[i] = np.vstack((obj_2_avevec[i], g[i].reshape(1,-1)))
    #     obj2[i] = np.vstack((np.array([[index_array[i]]]), obj_2_avevec[i]))    
    #     obj2[i] = np.vstack((np.array(obj2[i]), projection2_norm[i]))
    #     obj2[i] = np.vstack((np.array(obj2[i]), mse2[i]))
    #     obj2[i] = np.vstack((np.array(obj2[i]), abs(obj_2_avevec[i][0]/obj_2_avevec[i][1] - 2/3)))
    #     obj2[i] = np.vstack((np.array(obj2[i]), np.linalg.norm(proj(obj_2_avevec[i]) - proj(True_projection), 'fro')))
    #     obj2[i] = np.vstack((np.array(obj2[i]), Mse1(X @ obj_2_avevec[i], y[0])))
    
    # for i in range(S):
    #     g[i] = abs(obj_2_avevec[i][0] / obj_2_avevec[i][1] - a1/a2)
    #     g1[i] = proj_error_2[i]
    #     g2[i] = Mse1(X @ obj_2_avevec[i], y[0])
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g[i].reshape(1,-1)))
    #     obj_2_avevec[i] = np.vstack((np.array([[index_array[i]]]), obj_2_avevec[i]))    
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g1[i]))
    #     obj_2_avevec[i] = np.vstack((obj_2_avevec[i], g2[i]))
    
    # Result2 = [g[0] - min(g), g1[0] - min(g1), g2[0] - min(g2)] 
    # Result2 = [x.item() for x in Result2]
    
    # print("error_1:",[x.item() for x in error_1])
    # print("error_2:",[x.item() for x in error_2])
    # print("proj_error_1:",[x.item() for x in proj_error_1]) 
    # print("proj_error_2:",[x.item() for x in proj_error_2])
    
    # print(obj_2_avevec)
    # print(obj_2)
    # print("Obj1")
    # exhi(obj1) 
    # print("Obj2")
    # exhi(obj_2_avevec)
    # print(Result1), print(Result2)
    # return obj_2_avevec
    print("Obj1 Method 1")
    exhi(exhi_1)
    # print("Obj1 Method 2")
    # exhi(exhi_2)
    # # print("Obj1 Method 3")
    # return error_1_3, proj_error_1_3, prediction_mse_1_3
    return error_1_1

In [65]:
ar_coeff = [[0.2, 0.3, 0.3, 0.2], [0.2, 0.3, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
TestAR5(ar_coeff, 2, 3, initial_value, 10000, 1000, 50, 10, 100)

Obj1 Method 1
\begin{tabular}{rrr}
\hline
 0 & 14.0099 & 1.05609  \\ \hline
 1 & 14.0137 & 0.982861 \\ \hline
 2 & 14.0183 & 0.983874 \\ \hline
 3 & 14.0239 & 0.984811 \\ \hline
 4 & 14.0289 & 0.985453 \\ \hline
 5 & 14.0327 & 0.985878 \\ \hline
 6 & 14.0353 & 0.986165 \\ \hline
 7 & 14.0371 & 0.986352 \\ \hline
 8 & 14.0384 & 0.986482 \\ \hline
 9 & 14.0392 & 0.986563 \\ \hline
\hline \hline
\end{tabular} \hline


[array([0.23835255]),
 array([0.23426188]),
 array([0.23484006]),
 array([0.23488487]),
 array([0.23494548]),
 array([0.23500488]),
 array([0.23505096]),
 array([0.23507496]),
 array([0.23509776]),
 array([0.23511164])]

In [66]:
ar_coeff = [[0.1, 0.1, 0.3, 0.2], [0.1, 0.1, 0.5, 0.8]]
initial_value = [[0,0],[0,0],[0,0],[0,0]]
TestAR5(ar_coeff, 2, 3, initial_value, 10000, 1000, 50, 10, 100)

Obj1 Method 1
\begin{tabular}{rrr}
\hline
 0 & 14.0314 & 1.56713 \\ \hline
 1 & 14.0632 & 1.65987 \\ \hline
 2 & 14.0735 & 1.67973 \\ \hline
 3 & 14.0789 & 1.69075 \\ \hline
 4 & 14.0817 & 1.69801 \\ \hline
 5 & 14.0877 & 1.70335 \\ \hline
 6 & 14.0898 & 1.70654 \\ \hline
 7 & 14.0908 & 1.70847 \\ \hline
 8 & 14.0915 & 1.70963 \\ \hline
 9 & 14.092  & 1.71037 \\ \hline
\hline \hline
\end{tabular} \hline


[array([10.48858351]),
 array([2.67586757]),
 array([1.81047263]),
 array([1.76614362]),
 array([2.03154686]),
 array([2.62981592]),
 array([1.95008801]),
 array([1.87559938]),
 array([1.84111142]),
 array([1.83054317])]